# Simulation d'une croissance de forêt
 Ce programme vise à reconstituer l'évolution d'une forêt face à un contexte hostile.
 
### Légende :
 Chaque cellule représente une portion de territoire, elle peut être :
   - vierge, (BLANC)
   - arborée (VERT)
   - sporifiée, (JAUNE)
   - en feu, (ROUGE)
   - calcinée, (GRIS)
   - fertile, (MARRON).

### Détails avancés :
Une cellule vierge peut devenir sporifiée (JAUNE) ou arborée (VERT).

 Une cellule arborée peut vieillir (6 codes couleur : VERT CLAIR -> VERT FONCE) ou s'enflammer (ROUGE) à cause d'un incendie de forêt ou d'un éclair (source des incendies).

 Une cellule sporifiée (JAUNE) peut devenir arborée (VERT) ou vierge (BLANC).

 Une cellule en feu (ROUGE) devient calcinée (GRIS) resp. vierge (BLANC) selon le type de cellules environnantes : arborées (VERT) resp. fertiles (MARRON). 

 Une cellule calcinée (GRIS) peut devenir fertile (MARRON) ou vierge (BLANC).

 Une cellule fertile (MARRON) peut devenir arborée (VERT) ou s'enflammer (ROUGE).

### Quelques phénomènes observables :

#### Général :
 - le feu (ROUGE), les spores (JAUNE) et les arbres (VERT) se propagent de proche en proche.

#### Incendies (ROUGE) :
 - les incendies sont plus dévastateurs sur les cellules fertiles (MARRON) ou contenant de vieux arbres (VERT FONCE).
 - les incendies de zones arborées (VERT) laissent des traînées calcinées (GRIS) derrière eux.
 - un front d'incendie étroit (ROUGE) est susceptible d'être stoppé par un groupe de cellules arborées jeunes (VERT CLAIR).

#### Zones calcinées (GRIS) :
 - les grandes zones calcinées se fertilisent (MARRON) d'abord très lentement, puis de plus en plus vite.
 - les cellule calcinées (GRIS) isolées redeviennent rapidement vierges (BLANC).

#### Zones arborées (VERT) :
 - les cellules arborées jeunes (VERT CLAIR) vieillissent plus vite que celles plus âgées (VERT FONCE).
 - les zones arborées s'étendent plus facilement aux cellules fertile (MARRON) qu'aux cellules vierges (BLANC).
 - un arbre isolé (VERT) ne propage ni forêt (VERT), ni spores (JAUNE).
 - un vieil arbre isolé (VERT FONCE) a de grandes chances d'être frappé par la foudre (ROUGE) !

#### Spores (JAUNE) :
 - les spores se déplacent, ce qui leur permet d'arborer (VERT) des cellules vierges ou fertiles (BLANC / MARRON) pourtant éloignées des zones arborées (VERT).

#### Zones fertiles :
 - les incendies (ROUGE) de zones fertiles (MARRON) laissent de vastes espaces vierges (BLANC) derrière eux, ce qui permet de maintenir le role des spores (JAUNE).
 - la végétation (VERT) peut renaître sans arbres (VERT) ni spores (JAUNE) dans les zones fertiles (MARRON) !

In [10]:
#!pip install numpy
import sys, math, random
import pygame
import pygame.draw
from pygame.font import *
import numpy as np

In [11]:
__screenSize__ = (450,450)#(900,900)#
__cellSize__ = 10 
__gridDim__ = tuple(map(lambda x: int(x/__cellSize__), __screenSize__))
__density__ = 3 

__colors__ = [(255,255,255),(255,0,0),(0,255,0),(0,200,0),(0,150,0),(0,100,0),(0,50,0),(0,0,0),(240,240,0),(200,200,200),(180,120,20)]

glidergun=[
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,2,2],
  [0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,2,2],
  [2,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [2,2,0,0,0,0,0,0,0,0,2,0,0,0,2,0,2,2,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

def getColorCell(n):
    return __colors__[n]

class Grid:
    _grid= None
    _gridbis = None
    _indexVoisins = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]

    def __init__(self):
        print("Creating a grid of dimensions " + str(__gridDim__))
        self._grid = np.zeros(__gridDim__, dtype='int8')
        self._gridbis = np.zeros(__gridDim__, dtype='int8')
        if False: # True to init with one block at the center
            self._grid[nx//2,ny//2] = 1
            self._grid[nx//2+1,ny//2] = 1
            self._grid[nx//2,ny//2+1] = 1
            self._grid[nx//2+1,ny//2+1] = 1
        elif False: # True to init with random values at the center
            nx, ny = __gridDim__
            mx, my = 20, 16
            ones = np.random.random((mx, my)) > 0.75
            self._grid[nx//2-mx//2:nx//2+mx//2, ny//2-my//2:ny//2+my//2] = ones
        else: # Else if init with glider gun

            a = np.fliplr(np.rot90(np.array(glidergun),3))
            nx, ny = __gridDim__
            mx, my = a.shape
            self._grid[nx//2-mx//2:nx//2+mx//2, ny//2-my//2:ny//2+my//2] = a

    def indiceVoisins(self, x,y):
        return [(dx+x,dy+y) for (dx,dy) in self._indexVoisins if dx+x >=0 and dx+x < __gridDim__[0] and dy+y>=0 and dy+y < __gridDim__[1]] 

    def voisins(self,x,y):
        return [self._grid[vx,vy] for (vx,vy) in self.indiceVoisins(x,y)]
        
    def sommeVoisinsEnFeu(self, x, y):
        return sum(1 for index in self.voisins(x,y) if index==1)

    def sommeVoisinsEnVie(self, x, y):
        return sum(1 for index in self.voisins(x,y) if 1<index<8)

    def sommeVoisinsSpores(self, x, y):
        return sum(1 for index in self.voisins(x,y) if index==8)

    def sommeVoisinsBrules(self, x, y):
        return sum(1 for index in self.voisins(x,y) if index==9)

    def sommeVoisinsFertiles(self, x, y):
        return sum(1 for index in self.voisins(x,y) if index==10)

    def sumEnumerate(self):
        return [(c, self.sommeVoisinsEnVie(c[0], c[1]),self.sommeVoisinsEnFeu(c[0], c[1]),self.sommeVoisinsSpores(c[0], c[1]),self.sommeVoisinsBrules(c[0], c[1]),self.sommeVoisinsFertiles(c[0], c[1])) for c, _ in np.ndenumerate(self._grid)]

    def drawMe(self):
        pass


class Scene:
    _mouseCoords = (0,0)
    _grid = None
    _font = None

    def __init__(self):
        pygame.init()
        self._screen = pygame.display.set_mode(__screenSize__)
        self._font = pygame.font.SysFont('Arial',25)
        self._grid = Grid()

    def drawMe(self):
        if self._grid._grid is None:
            return
        self._screen.fill((255,255,255))
        for x in range(__gridDim__[0]):
            for y in range(__gridDim__[1]):
                pygame.draw.rect(self._screen, 
                        getColorCell(self._grid._grid.item((x,y))),
                        (x*__cellSize__ + 1, y*__cellSize__ + 1, __cellSize__-2, __cellSize__-2))


    def drawText(self, text, position, color = (255,64,64)):
        self._screen.blit(self._font.render(text,1,color),position)

    def updateForest(self):
        for c, voisinsEnVie, voisinsEnFeu, voisinsAvecSpores, voisinsBrules, voisinsFertiles in self._grid.sumEnumerate():
            cellType = self._grid._grid[c[0],c[1]]
            rand100 = random.randint(0,100)
            rand1K = random.randint(0,1000)

            # cell is empty -> can grow a tree or receive spores
            if cellType==0:                
                if voisinsEnFeu == 0 and voisinsEnVie > 1:
                    # cell can grow tree  : the more trees around the more likely
                    if rand100 < voisinsEnVie:
                        ret = 2

                    # cell receives spores from adjacent trees
                    elif rand100 < 5*voisinsEnVie and voisinsEnVie<5:
                        ret=8
                    else:
                        ret=0
                
                # spores can propagate further, split or disappear
                elif voisinsAvecSpores==1 and voisinsEnVie < 3:
                    if rand100 < 25 and voisinsEnFeu==0:
                        ret=8                  
                    else:
                        ret=0
                else:
                    ret=0

            # tree is on fire -> tree dies and becomes burned OR cell is cleared
            elif cellType == 1:
                if voisinsFertiles > 0:
                    ret=0
                else:
                    ret = 9

            # tree is alive -> tree can burn OR grow
            elif 1<cellType<8:
                
                # fire has high chances to spread (older trees are even more vulnerable)
                if voisinsEnFeu > 0 and rand100 < 4*cellType + 25*voisinsEnFeu:
                    ret=1

                # lightenings can set fire to trees (older/isolated ones are more vulnerable)
                elif (voisinsEnVie>0 and rand1K < cellType) or (voisinsEnVie==0 and rand1K < 4*cellType**2):
                    ret=1
                
                # trees can grow (older trees less likely)
                elif 1 < cellType < 7 and rand100 < 10/cellType:
                    ret=cellType+1

                # trees can simply stay alive
                else:
                    ret=cellType

            # cell contains spores -> cell can grow tree OR become empty
            elif cellType==8:
                if rand100 < 10 and voisinsEnVie < 3:
                    ret=2
                else:
                    ret=0
            
            # cell is burned -> cell can become fertile OR emptied
            elif cellType==9:
                if random.randint(0,1e4) < voisinsBrules**3 or (voisinsFertiles > 0 and rand100 < 5*voisinsFertiles):
                    ret=10
                elif voisinsBrules<4 and rand100 < 20 - 2*voisinsBrules - 2*voisinsFertiles:
                    ret=0
                else:
                    ret=9

            # cell is fertile -> cell has (higher) chances to grow tree OR become empty
            elif cellType == 10:

                # cell can grow tree
                if (voisinsEnVie>0 and rand100 < 2*voisinsEnVie or rand100<10) and voisinsBrules==0:
                    ret=2

                # cell meeting spores can grow tree
                elif rand100<20 and voisinsAvecSpores>1:
                    ret=2

                # cell can be burned to the ground
                elif voisinsEnFeu>0:
                    ret=1
                
                # cell can stay fertile
                else:
                    ret=10

            # nothing may happen too ... hey, why not ?!
            else:
                ret=cellType

            self._grid._gridbis[c[0], c[1]] = ret
        self._grid._grid = np.copy(self._grid._gridbis)

    def eventClic(self, coord, b):
        pass

    def recordMouseMove(self, coord):
        pass


In [12]:
def main():
    scene = Scene()
    done = False
    clock = pygame.time.Clock()
    while done == False:
        scene.drawMe()
        pygame.display.flip()
        scene.updateForest()
        clock.tick(2)
        for event in pygame.event.get():
            if event.type == pygame.QUIT: 
                print("Exiting")
                done=True
    pygame.quit()

if not sys.flags.interactive:
    main()

Creating a grid of dimensions (45, 45)
Exiting
